Replace placeholder paths like "book.jpg" and "video_first_frame.jpg" with actual file paths.

For the final AR video, loop over all frames in the video and apply the same logic.

You’ll need to use cv2.VideoCapture and cv2.VideoWriter for full AR video generation—let me know if you want help with that too.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# ================================
# Part 1: Augmented Reality
# ================================

def get_sift_matches(img1, img2, max_matches=50):
    """Detect keypoints using SIFT and find good matches using KNN + Lowe's ratio test."""
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)
            if len(good) >= max_matches:
                break

    pts1 = np.float32([kp1[m.queryIdx].pt for m in good])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good])

    return pts1, pts2, kp1, kp2, good


def compute_homography(src_pts, dst_pts):
    """Compute the homography matrix from source to destination points."""
    A = []
    for i in range(len(src_pts)):
        x, y = src_pts[i]
        u, v = dst_pts[i]
        A.extend([
            [-x, -y, -1, 0, 0, 0, x*u, y*u, u],
            [0, 0, 0, -x, -y, -1, x*v, y*v, v]
        ])
    A = np.asarray(A)
    U, S, Vt = np.linalg.svd(A)
    H = Vt[-1].reshape(3, 3)
    return H / H[2, 2]


def warp_perspective(img, H, shape):
    """Warp an image using the given homography matrix."""
    return cv2.warpPerspective(img, H, shape)


def overlay_image(background, overlay, mask):
    """Overlay image with a binary mask."""
    return np.where(mask[:, :, None], overlay, background)


def create_ar_frame(book_frame, ar_frame, H):
    """Overlay ar_frame onto book_frame using homography."""
    h_book, w_book = book_frame.shape[:2]
    ar_resized = cv2.resize(ar_frame, (w_book, h_book))
    warped_ar = cv2.warpPerspective(ar_resized, H, (w_book, h_book))

    mask = (warped_ar > 0).any(axis=2)
    return overlay_image(book_frame, warped_ar, mask)


In [ ]:
# ================================
# Part 2: Image Mosaics
# ================================

def warp_image_mosaic(img1, img2, H):
    """Warp img1 into img2's plane to create a mosaic."""
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    # Corners of the image to warp
    corners = np.float32([
        [0, 0],
        [0, h1],
        [w1, h1],
        [w1, 0]
    ]).reshape(-1, 1, 2)
    warped_corners = cv2.perspectiveTransform(corners, H)

    all_corners = np.concatenate((warped_corners, np.float32([
        [0, 0], [0, h2], [w2, h2], [w2, 0]
    ]).reshape(-1, 1, 2)), axis=0)

    [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
    translation = [-xmin, -ymin]

    result_shape = (xmax - xmin, ymax - ymin)
    H_translate = np.array([
        [1, 0, translation[0]],
        [0, 1, translation[1]],
        [0, 0, 1]
    ])

    # Warp first image
    warped_img1 = cv2.warpPerspective(img1, H_translate @ H, result_shape)
    # Paste second image
    result = warped_img1.copy()
    result[translation[1]:translation[1]+h2, translation[0]:translation[0]+w2] = img2

    return result

In [ ]:
# ================================
# Utility: Visualize Matches
# ================================

def draw_matches(img1, img2, kp1, kp2, matches):
    """Visualize matches between two images."""
    matched_img = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=2)
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(matched_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Feature Matches")
    plt.show()


In [ ]:
# ================================
# Example Usage
# ================================

if __name__ == "__main__":
    # Replace these with your actual image paths
    img_book = cv2.imread("book.jpg")
    video_frame = cv2.imread("video_first_frame.jpg")
    ar_frame = cv2.imread("ar_frame.jpg")
    img1 = cv2.imread("mosaic1.jpg")
    img2 = cv2.imread("mosaic2.jpg")

    # Part 1: AR
    pts_book, pts_frame, kp1, kp2, matches = get_sift_matches(img_book, video_frame)
    H = compute_homography(pts_book, pts_frame)
    draw_matches(img_book, video_frame, kp1, kp2, matches)
    ar_result = create_ar_frame(video_frame, ar_frame, H)
    cv2.imwrite("ar_overlay_result.jpg", ar_result)

    # Part 2: Image Mosaic
    pts1, pts2, _, _, _ = get_sift_matches(img1, img2)
    H_mosaic = compute_homography(pts1, pts2)
    mosaic = warp_image_mosaic(img1, img2, H_mosaic)
    cv2.imwrite("mosaic_result.jpg", mosaic)
